In [113]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [114]:
template_string = """Translate the text that is inside triple backticks into {style}. text: ```{text}```"""

In [115]:
from langchain.prompts import ChatPromptTemplate

promt_template = ChatPromptTemplate.from_template(template_string)

In [116]:
promt_template.messages[0].input_variables

['style', 'text']

In [117]:
custom_style = "An otaku style"

In [118]:
custom_text = "The ice-cream truck is so far away and it asks for a 1000 rupees! So damn expensive!"

In [119]:
custom_messages = promt_template.format_messages(
        style=custom_style,
        text=custom_text,
)

In [120]:
print(type(custom_text))

<class 'str'>


In [121]:
print(type(custom_text[0]))

<class 'str'>


In [122]:
response = model(custom_messages)
print(response.content)

```Ah, gosh darn it! The ice cream truck is, like, totally out of reach, man! And it's asking for a whoppin' 1000 rupees?! Like, are you kiddin' me?! That's, like, soooo darn expensive! I mean, I know it's not every day you see an ice cream truck, but come on! Can't they, like, lower the price or somethin'?! I'm talkin' serious cash, dude! 1000 rupees is like, a whole lotta yen! I mean, I'd have to, like, sell my soul to the otaku gods to get that kind of cash!


In [123]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [139]:
content_schema = ResponseSchema(name="content", description="The bdy of the response.")
references_schema = ResponseSchema(name="references", description="The references used to generate the response with links.")

In [140]:
response_schemas = [content_schema, references_schema]

In [141]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [142]:
format_instructions = output_parser.get_format_instructions()

In [143]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"content": string  // The bdy of the response.\n\t"references": string  // The references used to generate the response with links.\n}\n```'

In [144]:
research_template = """Read about {topic} and list 5 interesting facts about it. {format_instructions}"""

In [145]:
prompt2 = ChatPromptTemplate.from_template(template=research_template)

In [146]:
research_topic = "The WannaCry Cyber Attack"

In [147]:
custom_query = prompt2.format_messages(
    topic=research_topic,
    format_instructions=format_instructions,
)

In [148]:
answer = model(custom_query)

In [149]:
print(answer.content)

```json
{
  "content": "
**The WannaCry Cyber Attack: 5 Interesting Facts**

### 1. **Global Impact**: WannaCry was a global cyber attack that affected over 200,000 computers in more than 150 countries. It was one of the most widespread cyber attacks in history.

### 2. **EternalBlue Exploit**: WannaCry used the EternalBlue exploit, a Windows vulnerability that was discovered by the NSA and leaked by the Shadow Brokers hacking group. This exploit allowed the malware to spread quickly across networks.

### 3. **Ransomware**: WannaCry was a ransomware attack, meaning that it encrypted files on infected computers and demanded payment in Bitcoin to restore access. The ransom demand was around $300-$600 per victim.

### 4. **National Health Service (NHS) Impact**: The WannaCry attack had a significant impact on the UK's National Health Service (NHS), with many hospitals and clinics forced to shut down their IT systems. This led to delays and cancellations of medical treatments.

### 5. **Pa

In [150]:
output_dictionary = output_parser.parse(answer.content)

In [151]:
output_dictionary.get("content")

"\n**The WannaCry Cyber Attack: 5 Interesting Facts**\n\n### 1. **Global Impact**: WannaCry was a global cyber attack that affected over 200,000 computers in more than 150 countries. It was one of the most widespread cyber attacks in history.\n\n### 2. **EternalBlue Exploit**: WannaCry used the EternalBlue exploit, a Windows vulnerability that was discovered by the NSA and leaked by the Shadow Brokers hacking group. This exploit allowed the malware to spread quickly across networks.\n\n### 3. **Ransomware**: WannaCry was a ransomware attack, meaning that it encrypted files on infected computers and demanded payment in Bitcoin to restore access. The ransom demand was around $300-$600 per victim.\n\n### 4. **National Health Service (NHS) Impact**: The WannaCry attack had a significant impact on the UK's National Health Service (NHS), with many hospitals and clinics forced to shut down their IT systems. This led to delays and cancellations of medical treatments.\n\n### 5. **Patch Wednesda

In [152]:
output_dictionary.get("references")

'https://en.wikipedia.org/wiki/WannaCry_ransomware_attack'